In [64]:
import os
import sys
import pandas
import numpy as np
import torch
import requests
from pprint import pprint

In [55]:
A = {}
A.update({"a":3})
A

{'a': 3}

In [56]:
def get_coin_data(coin_id="archangel-token",contract="0x36e43065e977bc72cb86dbd8405fae7057cdc7fd"):
    # URL of the CoinGecko API for the Archangel Token contract details
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/contract/{contract}"

    # Sending a GET request to the URL
    response = requests.get(url)

    # Checking if the request was successful
    if response.status_code == 200:
        # Convert the JSON response into a Python dictionary
        data = response.json()
        data.update({"response":True,"gecko_id":coin_id,"contract":contract})
    else:
        data = {"response":False,"gecko_id":coin_id,"contract":contract}

    return data

In [95]:
from dataclasses import dataclass, fields,asdict
from typing import Optional

@dataclass
class PriceChangeData:
    #SENTIMENT
    sentiment_votes_up_percentage:Optional[int] = None
    watchlist_portfolio_users:Optional[int] = None
    market_cap_rank:Optional[int] = None
    #MARKET VALUES
    price_change_percentage_24h: Optional[float] = None
    price_change_percentage_7d: Optional[float] = None
    price_change_percentage_14d: Optional[float] = None
    price_change_percentage_30d: Optional[float] = None
    price_change_percentage_60d: Optional[float] = None
    price_change_percentage_200d: Optional[float] = None
    price_change_percentage_1y: Optional[float] = None
    price_change_percentage_1h_in_currency: Optional[float] = None
    price_change_percentage_24h_in_currency: Optional[float] = None
    price_change_percentage_7d_in_currency: Optional[float] = None
    price_change_percentage_14d_in_currency: Optional[float] = None
    price_change_percentage_30d_in_currency: Optional[float] = None
    price_change_percentage_60d_in_currency: Optional[float] = None
    price_change_percentage_200d_in_currency: Optional[float] = None
    price_change_percentage_1y_in_currency: Optional[float] = None
    current_price: Optional[float] = None
    total_value_locked: Optional[float] = None
    mcap_to_tvl_ratio: Optional[float] = None
    market_cap: Optional[float] = None
    market_cap_rank: Optional[int] = None
    uniswap:Optional[bool] = False

def obtain_tickers(data_tickers):
    bid_ask_spread_percentage = None
    uniswap = False
    if isinstance(data_tickers,list):
        for ticker in data_tickers:
            if isinstance(ticker,dict):
                if "market" in ticker.keys():
                    if 'uniswap' in ticker["market"]["identifier"]:
                        uniswap = True
                bid_ask_spread_percentage = ticker['bid_ask_spread_percentage']

    return {"uniswap":uniswap,
            "bid_ask_spread_percentage":bid_ask_spread_percentage}
                    
def prepare_dict_for_dataclasss(data):
    data_dict = {}
    if "market_data"in data.keys():
        data_dict.update(data["market_data"])
    if "tickers" in data.keys():
        tickers_data = obtain_tickers(data["tickers"])
        data_dict.update(tickers_data)
    return data_dict

def filter_dict_for_dataclass(input_dict, dataclass_type,currency="usd"):
    dataclass_fields = {f.name for f in fields(dataclass_type)}
    filtered_dict = {}
    for k, v in input_dict.items():
        if k in dataclass_fields:
            if not isinstance(v,dict):
                filtered_dict[k] = v
            else:
                if currency in v.keys():
                    filtered_dict[k] = v[currency] 
    return filtered_dict

In [96]:
import bdp

In [94]:
data_dict = prepare_dict_for_dataclasss(data)
filter_data_dict = filter_dict_for_dataclass(data_dict, PriceChangeData)

pprint(asdict(PriceChangeData(**filter_data_dict)))

{'current_price': 1.75e-12,
 'market_cap': 74056,
 'market_cap_rank': 3780,
 'mcap_to_tvl_ratio': None,
 'price_change_percentage_14d': 25.61798,
 'price_change_percentage_14d_in_currency': 25.61798,
 'price_change_percentage_1h_in_currency': None,
 'price_change_percentage_1y': -84.1055,
 'price_change_percentage_1y_in_currency': -84.1055,
 'price_change_percentage_200d': -87.36323,
 'price_change_percentage_200d_in_currency': -87.36323,
 'price_change_percentage_24h': None,
 'price_change_percentage_24h_in_currency': None,
 'price_change_percentage_30d': 58.0193,
 'price_change_percentage_30d_in_currency': 58.0193,
 'price_change_percentage_60d': 73.08124,
 'price_change_percentage_60d_in_currency': 73.08124,
 'price_change_percentage_7d': 8.71889,
 'price_change_percentage_7d_in_currency': 8.71889,
 'sentiment_votes_up_percentage': None,
 'total_value_locked': None,
 'uniswap': True,
 'watchlist_portfolio_users': None}


In [53]:
data = get_coin_data(coin_id="archangel-token",contract="0x36e43065e977bc72cb86dbd8405fae7057cdc7fd")

In [49]:
price_percentage_keys = ['price_change_percentage_24h',
 'price_change_percentage_7d',
 'price_change_percentage_14d',
 'price_change_percentage_30d',
 'price_change_percentage_60d',
 'price_change_percentage_200d',
 'price_change_percentage_1y',
 'price_change_percentage_1h_in_currency',
 'price_change_percentage_24h_in_currency',
 'price_change_percentage_7d_in_currency',
 'price_change_percentage_14d_in_currency',
 'price_change_percentage_30d_in_currency',
 'price_change_percentage_60d_in_currency',
 'price_change_percentage_200d_in_currency',
 'price_change_percentage_1y_in_currency']

In [88]:
data["market_data"]['price_change_percentage_14d_in_currency']["usd"]
data["market_data"]['current_price']["usd"]
data["market_data"]['total_value_locked']
data["market_data"]['mcap_to_tvl_ratio']
data["market_data"]['market_cap']["usd"]
data["market_data"]['market_cap_rank']

data["sentiment_votes_up_percentage"]
data["watchlist_portfolio_users"]
data["market_cap_rank"]

data["tickers"][0]["market"]["identifier"]

'uniswap_v2'

In [90]:
data["tickers"][0]['bid_ask_spread_percentage']

0.617689

3781

'uniswap_v2'

0.617689

In [44]:
data["tickers"][0].keys()

dict_keys(['base', 'target', 'market', 'last', 'volume', 'converted_last', 'converted_volume', 'trust_score', 'bid_ask_spread_percentage', 'timestamp', 'last_traded_at', 'last_fetch_at', 'is_anomaly', 'is_stale', 'trade_url', 'token_info_url', 'coin_id', 'target_coin_id'])

In [45]:
data["market_data"].keys()

dict_keys(['current_price', 'total_value_locked', 'mcap_to_tvl_ratio', 'fdv_to_tvl_ratio', 'roi', 'ath', 'ath_change_percentage', 'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'market_cap', 'market_cap_rank', 'fully_diluted_valuation', 'market_cap_fdv_ratio', 'total_volume', 'high_24h', 'low_24h', 'price_change_24h', 'price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d', 'price_change_percentage_200d', 'price_change_percentage_1y', 'market_cap_change_24h', 'market_cap_change_percentage_24h', 'price_change_24h_in_currency', 'price_change_percentage_1h_in_currency', 'price_change_percentage_24h_in_currency', 'price_change_percentage_7d_in_currency', 'price_change_percentage_14d_in_currency', 'price_change_percentage_30d_in_currency', 'price_change_percentage_60d_in_currency', 'price_change_percentage_200d_in_currency', 'price_change_percentage_1y_in_currency', 'market_cap_change_2

1.75e-12 None None 74056 3781


3398

3781